In [1]:
#set up
import pandas as pd
import requests
import json
from datetime import datetime
import sqlalchemy as sa 

#import env variables
from dotenv import load_dotenv
load_dotenv()

import os
airlab_api = os.environ['AIRLABS_API_KEY']

#get connection
from gans_connection import get_database_connection


#### lets create a table for airports infos



In [2]:
def fetch_airport_data(latitude, longitude):
    
    try:
        api_url = f'https://airlabs.co/api/v9/nearby?lat={latitude}&lng={longitude}&distance=2&lang=en&api_key={airlab_api}'
        response = requests.get(api_url)
        if response.status_code == 200:
            data = response.json()
            return data
        
    except Exception as e:
        print(f"Failed to fetch X.X = {response.status_code}")
        return None

In [3]:
def airports():
    try:
        engine = get_database_connection()
        cities_table = pd.read_sql('cities', engine)

        airports = []

        # Iterate through the cities and fetch airport data for each
        for index, row in cities_table.iterrows():
            city_id = row['city_id']
            latitude = row['latitude']
            longitude = row['longitude']

           
            data = fetch_airport_data(latitude, longitude)
            

            if data is not None:
                for r in data['response']['airports']:
                    current_time = datetime.now()
                    timestamp = current_time.strftime('%d-%m-%Y %H:%M:%S')

                    airports_info = {
                        "city_id": city_id,
                        "airport_name": r['name'],
                        "airport_iata": r['iata_code'],
                    }

                    airports.append(airports_info)

        # Process and store airport data
        airport_df = pd.DataFrame(airports)
        airport_df.dropna(subset=['airport_iata'], inplace=True)
        
        #select col for aiport table
        airport = airport_df[['airport_iata', 'airport_name']]
        
        # Select col for cities airport
        cities_airport = airport_df[['city_id', 'airport_iata']]

        # Store airport data in database
        #airport.to_sql('airports', engine, if_exists='append', index=False)
        
        #cities_airport.to_sql('cities_airports', engine, if_exists='append', index=False)
        print(airport)
        print(cities_airport)
        return print("Flying :D ")
        
    except Exception as e:
        print(f"Uh-oh, unexpected AIRPORT anomaly X.X {e}")
        
   


In [4]:
airports()

  airport_iata                     airport_name
0          GVA         Aeroport Geneve-Cointrin
1          LIS  Lisbon Humberto Delgado Airport
   city_id airport_iata
0        1          GVA
1        2          LIS
Flying :D 
